In [1]:
vals = [1, 2, 3]

In [2]:
vals[-2:]

[2, 3]

In [ ]:
class MKAverage:

    def __init__(self, m: int, k: int):
        self.m = m
        self.k = k
        self.nums = []
        

    def addElement(self, num: int) -> None:
        self.nums.append(num)

    def calculateMKAverage(self) -> int:
        if len(self.nums) < self.m:
            return -1
        
        vals = self.nums[-self.m:]
        vals.sort()
        sub_vals = vals[self.k:-self.k]
        return sum(sub_vals) // len(sub_vals)

In [1]:
from sortedcontainers import SortedList
from collections import defaultdict, deque
import bisect

class MKAverage:
    def __init__(self, m: int, k: int):
        self.m = m
        self.k = k
        self.nums = deque()
        self.s_nums = SortedList() # 经过排序的数字
        self.total = 0 # 最后 m 个元素的和
        self.topKMin = 0 # m 个元素中 最小 k 个值的和
        self.topKMax = 0 # m 个元素中 最大 k 个值的和
        
    def addElement(self, num: int) -> None:
        self.total += num
        self.nums.append(num)
        idx = self.s_nums.bisect_left(num)
        
        if idx < self.k:
            self.topKMin += num
            # 因为如果 self.s_nums 的长度是小于 k 的，那么 top_k_min 和 top_k_max是相同的
            # 如果 最后 m 个数超过了 self.k, 那么需要将 之前被算作是 min_val 中减去
            if len(self.s_nums) >= self.k:
                self.topKMin -= self.s_nums[self.k-1]
        # 同理
        if idx >= len(self.s_nums) + 1 - self.k:
            self.topKMax += num
            if len(self.s_nums) >= self.k:
                self.topKMax -= self.s_nums[-self.k]
                
        # 加入自动排序的  s_nums
        self.s_nums.add(num)
        
        # 加入的数字中超过了 self.m
        if len(self.s_nums) > self.m:
            # 弹出最先添加的元素
            num_to_remove = self.nums.popleft()
            self.total -= num_to_remove
            idx = self.s_nums.index(num_to_remove) # 查看所在位置
            
            if idx < self.k:
                # 属于 top-k-min, 减去之后，再加上一个数字
                self.topKMin -= num_to_remove
                self.topKMin += self.s_nums[self.k]
            elif idx >= len(self.s_nums) - self.k:
                self.topKMax -= num_to_remove
                self.topKMax += self.s_nums[-self.k-1]
            self.s_nums.remove(num_to_remove)
            
    def calculateMKAverage(self) -> int:
        if len(self.nums) < self.m:
            return -1
        return (self.total - self.topKMax - self.topKMin) // (self.m - self.k * 2)